In [1]:
%matplotlib inline

In [2]:
path = "data/sign_short_preprocess/"

In [3]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

In [4]:
import utils; reload(utils)
from utils import plots

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 850M (CNMeM is disabled, cuDNN not available)
Using Theano backend.


In [5]:
batch_size=32

In [6]:
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [7]:
# The core part
vgg = Vgg16()
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
# vgg.model.load_weights('vgg16_sign1.h5')
vgg.finetune(batches)
# vgg.model.load_weights('vgg16_sign_exp1_custom_background.h5')
vgg.model.optimizer.lr = 0.001

Found 292 images belonging to 4 classes.
Found 28 images belonging to 4 classes.


In [8]:
# The fitting part (Expensive to execute)
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
292/292 [==============================] - 26s - loss: 2.1129 - acc: 0.4384 - val_loss: 0.6674 - val_acc: 0.6786


In [15]:
# The testing part
batches, res = vgg.test(path+'test/')
for i, x in enumerate(res[0]):
    print(vgg.classes[i],x)
print(max(res[0]))
vgg.classes[res[0].argmax(axis=0)]

Found 1 images belonging to 1 classes.
a 0.0505342
b 0.854229
m 0.0540058
y 0.0412309
0.854229


'b'

In [12]:
vgg.model.optimizer.lr = 0.0001
vgg.fit(batches, val_batches, nb_epoch=3)

Epoch 1/3
292/292 [==============================] - 25s - loss: 1.5245 - acc: 0.4966 - val_loss: 0.4325 - val_acc: 0.8214
Epoch 2/3
292/292 [==============================] - 25s - loss: 1.1687 - acc: 0.6473 - val_loss: 0.2771 - val_acc: 0.9286
Epoch 3/3
292/292 [==============================] - 25s - loss: 0.9273 - acc: 0.6884 - val_loss: 0.2645 - val_acc: 0.8929


In [22]:
res[0].argmax(axis=0)

2

In [14]:
vgg.model.save_weights('vgg16_sign_short_exp1.h5')

In [28]:
vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [8]:
vgg.model.optimizer.lr = 0.002